In [14]:
from loadData import loadData
from extract_fiducials import extract_fiducials
from extract_features import extract_features
from preprocess import preprocess

import numpy as np

# Load Data

In [2]:
ROOT_PATH = u'C:/Users/cricr/OneDrive - Politecnico di Milano/Università/MASTER/applied AI in Biomedicine/project/train'

In [3]:
ppgs_128, speaks_128, annotations_128, ppgs_250, speaks_250, annotations_250 = loadData(ROOT_PATH)

Loading file: 1/62
Loading file: 2/62
Loading file: 3/62
Loading file: 4/62
Loading file: 5/62
Loading file: 6/62
Loading file: 7/62
Loading file: 8/62
Loading file: 9/62
Loading file: 10/62
Loading file: 11/62
Loading file: 12/62
Loading file: 13/62
Loading file: 14/62
Loading file: 15/62
Loading file: 16/62
Loading file: 17/62
Loading file: 18/62
Loading file: 19/62
Loading file: 20/62
Loading file: 21/62
Loading file: 22/62
Loading file: 23/62
Loading file: 24/62
Loading file: 25/62
Loading file: 26/62
Loading file: 27/62
Loading file: 28/62
Loading file: 29/62
Loading file: 30/62
Loading file: 31/62
Loading file: 32/62
Loading file: 33/62
Loading file: 34/62
Loading file: 35/62
Loading file: 36/62
Loading file: 37/62
Loading file: 38/62
Loading file: 39/62
Loading file: 40/62
Loading file: 41/62
Loading file: 42/62
Loading file: 43/62
Loading file: 44/62
Loading file: 45/62
Loading file: 46/62
Loading file: 47/62
Loading file: 48/62
Loading file: 49/62
Loading file: 50/62
Loading f

# Pre-processing

In [4]:
for i in range(len(ppgs_128)):
    ppgs_128[i] = preprocess(ppgs_128[i])

In [5]:
for i in range(len(ppgs_250)):
    ppgs_250[i] = preprocess(ppgs_250[i])

# Extract fiducials

In [6]:
fiducials_128 = list()

for i in range(len(ppgs_128)):
    print(i)
    fiducials_128.append(extract_fiducials(ppgs_128[i], speaks_128[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [7]:
fiducials_250 = list()

for i in range(len(ppgs_250)):
   print(i)
   fiducials_250.append(extract_fiducials(ppgs_250[i], speaks_250[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


# Extract features

In [8]:
features_128 = list()

for i in range(len(fiducials_128)):
    features_128.append(extract_features(fiducials_128[i], ppgs_128[i], 128))

In [9]:
features_250 = list()

for i in range(len(fiducials_250)):
    features_250.append(extract_features(fiducials_250[i], ppgs_250[i], 250))

# Add annotation to peaks

In [10]:
for i in range(len(features_128)):
    features_128[i]['annotation'] = annotations_128[i]

In [11]:
for i in range(len(features_250)):
    features_250[i]['annotation'] = annotations_250[i]

# Remove rows with null values

In [15]:
for i in range(len(features_128)):
    # Find rows with at least one NaN value using np.isnan()
    rows_with_nan = features_128[i][features_128[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]

    # Remove rows with NaN values
    features_128[i] = features_128[i].drop(rows_with_nan.index)

C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\510882841.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  rows_with_nan = features_128[i][features_128[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]
C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\510882841.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  rows_with_nan = features_128[i][features_128[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]
C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\510882841.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  rows_with_nan = features_128[i][features_128[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]
C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\510882841.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use

In [16]:
for i in range(len(features_250)):
    # Find rows with at least one NaN value using np.isnan()
    rows_with_nan = features_250[i][features_250[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]

    # Remove rows with NaN values
    features_250[i] = features_250[i].drop(rows_with_nan.index)

C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\3240610789.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  rows_with_nan = features_250[i][features_250[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]
C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\3240610789.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  rows_with_nan = features_250[i][features_250[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]
C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\3240610789.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  rows_with_nan = features_250[i][features_250[i].applymap(lambda x: np.isnan(x) if isinstance(x, (float, np.float64)) else False).any(axis=1)]
C:\Users\cricr\AppData\Local\Temp\ipykernel_10760\3240610789.py:3: FutureWarning: DataFrame.applymap has been deprecated.

# Write features to csv

In [28]:
i = 0
for feature in features_128:
    file_name = f"C:/Users/cricr/PycharmProjects/AI-project/features/128/features_{i}_128.csv"
    feature.to_csv(file_name, sep=',', index=False, encoding='utf-8')
    i = i + 1

In [29]:
i = 0
for feature in features_250:
    file_name = f"C:/Users/cricr/PycharmProjects/AI-project/features/250/features_{i}_250.csv"
    feature.to_csv(file_name, sep=',', index=False, encoding='utf-8')
    i = i + 1